
**[PT]** Para executar este ficheiro veja o ficheiro README.md nesta directoria

**[EN]** To run this file: follow instructions in the README.md file in this directory.

# **[PT]** Correção, inferência e normalização do campo "Faculdade"

# **[EN]** Correction, inference and normalization of the field "Faculdade"


## Setup

In [25]:
from timelinknb import current_time,current_machine, get_mhk_db
import ucalumni.config as alumniconf

db_name = alumniconf.mhk_db_name
db = get_mhk_db(db_name)
print(current_machine,current_time,f'db={db_name}')

imac-jrc.local 2022-05-10 14:24:39.168529 db=ucalumni


## **[PT]** Listagens de controle
## **[EN]** Control listings

### **[PT]** Notas de processamento


In [82]:
from timelinknb.pandas import attribute_to_df

notas_proc = attribute_to_df(
                    the_type='nota-processamento',
                    person_info=True,
                    more_cols=['faculdade','faculdade-original','uc-entrada', 'uc-saida','url'],
                    sql_echo=False)
                    


In [85]:
notas_proc['id'] = notas_proc.index.values
notas_proc.dropna(how='all', axis=1, inplace=True)
notas_proc.info()

In [86]:
df = notas_proc
df['id'] = df.index.values

col = 'nota-processamento' # sub total by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc-entrada':'min',
                                                  'uc-saida':'max'})
df_totals.loc['Total'] = df_totals['id'].sum()

df_totals.sort_values('id',ascending= False).head(30)

,id,uc-entrada,uc-saida
nota-processamento,,,
Total,20263,20263,20263
Aviso: faculdade inferida,11356,0000-00-00,1913-07-11
Aviso: faculdade corrigida,6980,0000-00-00,1916-07-15
"Faculdade corrigida, combinação incomum",735,0000-00-00,1916-07-15
Erro: não foi possível determinar a faculdade,675,1537-00-00,1912-08-02
Erro: valor não corresponde a Faculdade nesta data,517,0000-00-00,1913-08-13


### **[EN]** "Faculdade" Inferred

In narrative terms, the algorithm checks first if the field “Faculdade” has a value or not; it then proceeds to collect all references to “faculdades” in the rest of the record; if by removing the pre-requisites from the collection something remains, it will the chosen as the corrected or inferred “faculdade”; if nothing remains then the original value is used; some other edge cases are tested. 


### **[PT]** "Faculdade" inferida



In [87]:
fac_inf = notas_proc[notas_proc['nota-processamento']=='Aviso: faculdade inferida'].copy()
fac_inf.dropna(how='all', axis=1, inplace=True)
fac_inf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11759 entries, 127775 to 319862
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   name                     11759 non-null  object
 1   sex                      11759 non-null  object
 2   nota-processamento       11759 non-null  object
 3   nota-processamento.date  11759 non-null  object
 4   nota-processamento.obs   11759 non-null  object
 5   faculdade                11759 non-null  object
 6   faculdade.date           11759 non-null  object
 7   faculdade.obs            11759 non-null  object
 8   uc-entrada               11759 non-null  object
 9   uc-entrada.date          11759 non-null  object
 10  uc-saida                 11759 non-null  object
 11  uc-saida.date            11759 non-null  object
 12  url                      11759 non-null  object
 13  url.date                 11759 non-null  object
 14  id                       11759 non-nu

In [88]:
df = fac_inf
col = 'faculdade' # subotal by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc-entrada':'min',
                                                  'uc-saida':'max'})
print("List of inferred 'faculdade'")
df_totals.sort_values('id',ascending= False).head(30)

List of inferred 'faculdade'


,id,uc-entrada,uc-saida
faculdade,,,
Cursos jurídicos (Cânones ou Leis),6993,0000-00-00,1829-03-28
Artes,3005,0000-00-00,1780-07-18
Cânones,764,0000-00-00,1827-06-20
Leis,386,0000-00-00,1828-07-01
Teologia,352,0000-00-00,1884-05-26
Medicina,185,0000-00-00,1883-10-15
Filosofia,28,1770-10-01,1913-07-11
Matemática,24,1605-10-07,1913-07-11
Direito,22,1836-10-14,1898-12-18


### **[PT]** Faculdade corrigida



In [129]:
fac_corr = notas_proc[notas_proc['nota-processamento']=='Aviso: faculdade corrigida'].copy()
fac_corr.dropna(how='all', axis=1, inplace=True)

In [148]:
import pandas as pd

df = fac_corr
col = 'nota-processamento.obs' # subotal by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc-entrada':'min',
                                                  'uc-saida':'max'})
total = df_totals['id'].sum()
print("List of corrected 'faculdade'")
print("Total:",total)
df_totals['perc'] = df_totals['id']/total
pd.set_option('display.max_rows',1000)
#df_totals.sort_values('id',ascending=False, inplace=True)
df_totals.head(10)

List of corrected 'faculdade'
Total: 6980


,id,uc-entrada,uc-saida,perc
nota-processamento.obs,,,,
"""Direito / Cânones para Cânones,Leis.""",1,1814-10-05,1822-10-31,0.000143
"""Matemática / Filosofia / Medicina para Medicina.""",1,1783-10-16,1791-07-30,0.000143
". Cânones para Cânones,Leis.",2,1577-10-29,1640-04-20,0.000287
". Cânones para Cânones,Teologia.",1,1684-10-01,1690-10-01,0.000143
". Leis para Cânones,Leis.",2,1574-10-01,1617-10-27,0.000287
". Medicina para Medicina,Teologia.",1,1617-10-13,1618-11-10,0.000143
". para (.),Cânones.",1,1633-10-15,1654-10-21,0.000143
". para (.),Medicina.",1,1658-10-01,1674-10-01,0.000143
". para (.),Teologia.",1,1738-12-01,1750-10-01,0.000143


In [133]:
df_totals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, Cânones para Cânones,Leis. to teologia para Cânones,Teologia.
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          127 non-null    int64 
 1   uc-entrada  127 non-null    object
 2   uc-saida    127 non-null    object
dtypes: int64(1), object(2)
memory usage: 4.0+ KB


In [149]:
fac_corr_nodup = fac_corr.drop_duplicates(subset=['id', 'faculdade-original'], keep='first')

In [150]:
for  change,row in list(df_totals.iterrows())[:10]:
    print(f"{change}: {row.id}")
    for _ndx, row in fac_corr_nodup[fac_corr_nodup['nota-processamento.obs']==change][['id','name','url','uc-entrada']].head(10).iterrows():
        url = row.url.strip('"')
        print(f"     {row.id:6} {row['name']} {row['uc-entrada']} {url}")


"Direito / Cânones para Cânones,Leis.": 1
     153043 António Alves de Carvalho 1814-10-05 https://pesquisa.auc.uc.pt/details?id=153043
"Matemática / Filosofia / Medicina para Medicina.": 1
     202402 Vicente Coelho da Silva Seabra e Teles 1783-10-16 https://pesquisa.auc.uc.pt/details?id=202402
. Cânones para Cânones,Leis.: 2
     165691 Manuel Ribeiro 1577-10-29 https://pesquisa.auc.uc.pt/details?id=165691
     165826 Manuel Ribeiro 1625-11-15 https://pesquisa.auc.uc.pt/details?id=165826
. Cânones para Cânones,Teologia.: 1
     165732 Manuel Ribeiro 1684-10-01 https://pesquisa.auc.uc.pt/details?id=165732
. Leis para Cânones,Leis.: 2
     165690 Manuel Ribeiro 1620-10-06 https://pesquisa.auc.uc.pt/details?id=165690
     166337 Manuel Ribeiro 1574-10-01 https://pesquisa.auc.uc.pt/details?id=166337
. Medicina para Medicina,Teologia.: 1
     165782 Manuel Ribeiro 1617-10-13 https://pesquisa.auc.uc.pt/details?id=165782
. para (.),Cânones.: 1
     165694 Manuel Ribeiro 1633-10-15 https://p

### **[PT]** "Valor não corresponde a Faculdade nesta data"

Esta lista é produzida pela comparação do valor do campo de faculdade, ou outra referência a faculdade no registo, e a lista das faculdades existentes à data do registo.

Para além dos erros de ortografia e gralhas (atenção a "Cãnones" com til), também contém:

* Uma grande quantidade (>300) de campos de Faculdade com "?". Verificar se é porque falta informação na ficha, ou se existe outra explicação
* valores confundidos com Faculdade por particularidades da pontuação do registo. Nesse caso a correção a fazer pode ser na pontuação. 
   Ver por exemplo: 152975 a falta de ":" a seguir a matrícula provoca um erro no processamento da Faculdade
* valor que correspondem a faculdades que existiram, mas não naquelas datas (por exemplo Filosofia antes de 1772 ou Cânones depois de 1834).
* Topónimos, que serão erros de digitação que se poderão corrigir vendo a ficha original.



In [37]:
fac_error = notas_proc[notas_proc['nota-processamento']=='Erro: valor não corresponde a Faculdade nesta data'].copy()
fac_error.info()

<class 'pandas.core.frame.DataFrame'>
Index: 519 entries, 130413 to 317873
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   name                     519 non-null    object
 1   sex                      519 non-null    object
 2   nota-processamento       519 non-null    object
 3   nota-processamento.date  519 non-null    object
 4   nota-processamento.obs   519 non-null    object
 5   faculdade                519 non-null    object
 6   faculdade.date           519 non-null    object
 7   faculdade.obs            3 non-null      object
 8   faculdade-original       206 non-null    object
 9   faculdade-original.date  206 non-null    object
 10  faculdade-original.obs   0 non-null      object
 11  uc-entrada               519 non-null    object
 12  uc-entrada.date          519 non-null    object
 13  uc-entrada.obs           0 non-null      object
 14  uc-saida                 519 non-null  

In [40]:
# currently the algorithm removed question marks, but they were there
# we have to put them back until this is fixed
fac_org_none = fac_error['faculdade-original'].isnull()
fac_error.loc[fac_org_none,'faculdade-original'] = '?'


In [43]:
df = fac_error
col = 'faculdade-original' # subotal by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc-entrada':'min',
                                                  'uc-saida':'max'})

df_totals.sort_values('id',ascending= False).head(30)

,id,uc-entrada,uc-saida
faculdade-original,,,
?,313,0000-00-00,1771-01-01
Leis,23,1827-10-13,1909-08-03
Cânones,22,1825-10-24,1913-08-13
Cãnones,21,1617-11-29,1769-07-10
Medecina,18,1654-10-15,1858-10-12
.,13,1604-10-17,1732-11-10
Intituta,8,1581-03-12,1770-10-01
Instituto,7,1694-12-15,1818-10-24
Medicna,5,1590-10-08,1735-05-08


In [42]:
df_totals['id'].sum()

517

In [10]:
import pandas as pd
import csv

pd.set_option('display.max_rows',600)
exportar = erro_faculdade[['faculdade','name','faculdade.date']].sort_values(['faculdade','name','faculdade.date'])
exportar.to_csv('../inferences/validation/faculdade-unkown.csv',sep=',')
exportar.head(100)

,faculdade,name,faculdade.date
id,,,
192086,('),António Rodrigues,1626-10-29
146645,(.),António Antunes,1656-10-20
140542,(.),António de Abreu,1661-10-26
165702,(.),Manuel Ribeiro,1604-10-17
165556,(.),Manuel Ribeiro,1612-03-23
165558,(.),Manuel Ribeiro,1617-04-09
165781,(.),Manuel Ribeiro,1657-10-15
165804,(.),Manuel Ribeiro,1659-12-12
165730,(.),Manuel Ribeiro,1683-03-24


### "Faculdades" com menos de 100 ocorrências. 

Uma parte são gralhas na digitação no nome da faculdade. 

Algumas poderiam ser corrigidas automaticamente, mas é preferível que o ficheiro original seja limpo destes erros.

Outros casos são provocados por erros na linha a seguir ao campo "Faculdade". 

Quando uma linha não se inicia por um identificador de campo, o programa tenta juntar o conteúdo dessa linha aos valores do último campo processado.  Sem essa operação não seria possível processar a informação de matrículas e exames, que repetem valores em linhas sucessivas, sem indicar o campo em cada linha.

Quando o campo "Matrícula(s)", que normalmente sucede "Faculdade", foi registado de forma deficiente, os valores das matrículas podem ser interpretados como dizendo respeito à faculdade.

Embora seja possível corrigir alguns destes erros automaticamente, é preferível normalizar o ficheiro original, verificando quais os erros da listagem abaixo se devem a deficiente registo do campo "Matrícula(s)".

### Faculdades com menos ocorrências

A lista abaixo inclui, por ordem alfabética, os valores do campo "Faculdade" com menos de 100 ocorrências.



In [9]:

from sqlalchemy import select,distinct,func,desc, and_
from ipywidgets import HTML
from timelinknb.config import Session     
from timelinknb.pandas import get_nattribute_table

# get the named attributes table
nattr_table = get_nattribute_table()

stmt = select(nattr_table.c.the_value, func.count(distinct(nattr_table.c.id)).label("n")).\
        where(nattr_table.c.the_type == 'faculdade').\
        group_by(nattr_table.c.the_value).having(func.count()<100).\
        order_by(nattr_table.c.the_value)

print()
#print("Using code tag")
#print()
with Session() as session:
        for row in session.execute(stmt):
                stmt2 = select(nattr_table.c.id,nattr_table.c.name, nattr_table.c.the_date, nattr_table.c.pobs, nattr_table.c.aobs).\
                                where(
                                        and_(nattr_table.c.the_type == 'faculdade', 
                                        nattr_table.c.the_value == row.the_value)).\
                                order_by(nattr_table.c.name)
                result = session.execute(stmt2)
                for aluno in result:
                        id_link=  f'<a href="https://pesquisa.auc.uc.pt/details?id={aluno.id}">{aluno.id}</a>'
                        # display(HTML(f'<code>{row.the_value:24} [{id_link:5}] {aluno.the_date} {aluno.name:44}</code>'))
                        obs = aluno.aobs if aluno.aobs is not None else ''
                        print(f'{row.the_value:24} [{aluno.id:5}] {aluno.the_date} {aluno.name:44} https://pesquisa.auc.uc.pt/details?id={aluno.id} {obs}')


(')                      [192086] 1626-10-29 António Rodrigues                            https://pesquisa.auc.uc.pt/details?id=192086 
(. )                     [165828] 1689-11-24 Manuel Ribeiro                               https://pesquisa.auc.uc.pt/details?id=165828 
(. )                     [165775] 1730-03-15 Manuel Ribeiro                               https://pesquisa.auc.uc.pt/details?id=165775 
(. )                     [165694] 1633-10-15 Manuel Ribeiro                               https://pesquisa.auc.uc.pt/details?id=165694 Instituta: 1632-10-15 1632.10.15
(. )                     [165753] 1658-10-01 Manuel Ribeiro                               https://pesquisa.auc.uc.pt/details?id=165753 1674.10.01
(. )                     [165702] 1604-10-17 Manuel Ribeiro                               https://pesquisa.auc.uc.pt/details?id=165702 
(. )                     [165804] 1659-12-12 Manuel Ribeiro                               https://pesquisa.auc.uc.pt/details?id=165804 
(. ) 

## Registos em mais que uma Faculdade

Quando não existe uma menção explícita de faculdade o programa usa a lista de matrículas, o que pode produzir uma lista de faculdades para um mesmo aluno.

Sobre as combinações de frequência de faculdades ver Fonseca, F.T. da (2000) ‘A dimensão pedagógica da reforma de 1772: alguns aspectos’, in O Marquês de Pombal e a Universidade. Coimbra: Imprensa da Universidade de Coimbra, pp. 43–68. doi:http://dx.doi.org/10.14195/978-989-26-0373-5_2.

Antes de 1772 a frequência de Artes precede a Teologia (122 casos), Medicina  (112) obrigatóriamente, mas também aparece antes de 
Cânones (94) e Leis  (38) e de Cânones e Leis (23), e outras combinações

Depois de 1772 a frequência de Filosofia e Matemática concide com a frequência de Direito, Cânones e Medicina.

As Faculdade de Matemática e Filosofia só deveriam ser referidas autónomamente a partir de 1772.


### Alunos com matrícula em mais que uma faculdade
### Students with more that one "Faculdade"


In [5]:
from sqlalchemy import select,distinct,func,desc, and_
# List of students with more than one record of faculdade
stmt = select(nattr_table.c.id, nattr_table.c.name, func.count().label("n")).\
        where(nattr_table.c.the_type == 'faculdade').\
        group_by(nattr_table.c.id, nattr_table.c.name).having(func.count()>1).\
        order_by(desc("n"))
print(stmt)
# we store each combination and number of ocurrences

combinacoes = {}

with Session() as session:
        for aluno in session.execute(stmt):
                id_link=f'<a href="https://pesquisa.auc.uc.pt/details?id={aluno.id}">{row.id}</a>'
                #display(HTML(f'<pre> [{id_link}]  {row.name} ({row.n})</pre>'))
                #print()
                stmt2 = select(nattr_table.c.the_date,nattr_table.c.the_value, nattr_table.c.pobs).\
                                where(
                                        and_(nattr_table.c.the_type == 'faculdade', 
                                        nattr_table.c.id == aluno.id)).distinct().\
                                order_by(nattr_table.c.the_value)
                result = session.execute(stmt2)
                combinacao = ''
                aluno_faculdades = []
                for fac in result:
                        #display(HTML(f'<pre>    {aluno.the_value} {aluno.the_date}</pre>'))
                        #print(f'   {aluno.the_date} {aluno.the_value} ')
                        aluno_faculdades.append((fac.the_value,fac.the_date))
                        combinacao = combinacao + ' ' + fac.the_value
                alunos = combinacoes.get(combinacao,{})
                alunos[aluno.id]={'name':aluno.name,'matriculas':aluno_faculdades}
                combinacoes[combinacao] = alunos


SELECT nattributes.id, nattributes.name, count(*) AS n 
FROM nattributes 
WHERE nattributes.the_type = :the_type_1 GROUP BY nattributes.id, nattributes.name 
HAVING count(*) > :count_1 ORDER BY n DESC


AttributeError: Could not locate column in row for column 'id'

A lista seguinte mostra as diferentes combinações de faculdades que ocorrem para um mesmo aluno. Quando o número de alunos uma combinação específica é inferior a um certo valor o nome dos alunos é listado, para verificação de eventuais erros.

As faculdades são combinadas por ordem alfabética, mas na listagem as informações referentes às matrículas são mostradas por ordem cronológica.

In [ ]:
limiar = 10
print("Limiar utilizado para listar os alunos de uma determinada combinação: ",limiar)
for comb,alunos in sorted(combinacoes.items()):
    print()
    print(f'{len(alunos):3} {comb}' )
    n = 0

    for nome,info in alunos.items():
        aluno_id = info['id']
        print(f'       {nome} https://pesquisa.auc.uc.pt/details?id={aluno_id}')
        for matricula,data in sorted(info['matriculas'], key = lambda x: x[1]):

            print(f'          {data} {matricula}')
        n = n + 1
        if n > limiar:
            print('       ...')
            break

### Aluno com '?' no campo Faculdade

Presume-se que estes casos correspondem aos que não foi possível determinar a faculdade do aluno.

De qualquer modo valeria a pena verificar se não há aqui, também, gralhas e cópias acidentais de valores entre registos.

In [ ]:
# Id, data, nome
# see https://docs.sqlalchemy.org/en/14/tutorial/data_select.html#tutorial-selecting-data
from sqlalchemy import select, and_, or_, func, desc
from IPython.core.display import display, HTML


#display(HTML("<h2>Faculdade = '?'</h2>"))

stmt = select(nattr_table.c.id,nattr_table.c.name, nattr_table.c.the_date, nattr_table.c.pobs).\
        where(
                and_(nattr_table.c.the_type == 'faculdade', 
                     nattr_table.c.the_value == '?')).\
        order_by(nattr_table.c.name)
result = session.execute(stmt)
for row in result:
    id_link=  f'<a href="https://pesquisa.auc.uc.pt/details?id={row.id}">{row.id}</a>'
    line =f'<pre>[{id_link:5}] {row.the_date} {row.name}</pre>'
    #display(HTML(line))
    print(f'{row.id:5} {row.the_date} {row.name:64} https://pesquisa.auc.uc.pt/details?id={row.id}')
